In [15]:
from pathlib import Path
from numpy.core.numeric import NaN
import numpy as np
import nibabel as nib
import nibabel.processing as nip
import nibabel.orientations as nio
from scipy.ndimage import center_of_mass
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap, Normalize
from matplotlib.patches import Circle
import json
import tools as tools

In [14]:
def reorient_to(img, axcodes_to=('P', 'I', 'R'), verb=False):
    """Reorients the nifti from its original orientation to another specified orientation
    
    Parameters:
    ----------
    img: nibabel image
    axcodes_to: a tuple of 3 characters specifying the desired orientation
    
    Returns:
    ----------
    newimg: The reoriented nibabel image 
    
    """
    aff = img.affine
    ornt_fr = nio.io_orientation(aff)
    axcodes_fr = nio.ornt2axcodes(ornt_fr)
    if axcodes_to == axcodes_fr:
        return img
    ornt_to = nio.axcodes2ornt(axcodes_to)
    arr = np.asanyarray(img.dataobj, dtype=img.dataobj.dtype)
    ornt_trans = nio.ornt_transform(ornt_fr, ornt_to)
    arr = nio.apply_orientation(arr, ornt_trans)
    aff_trans = nio.inv_ornt_aff(ornt_trans, arr.shape)
    newaff = np.matmul(aff, aff_trans)
    newimg = nib.Nifti1Image(arr, newaff)
    if verb:
        print("[*] Image reoriented from", nio.ornt2axcodes(ornt_fr), "to", axcodes_to)
    return newimg


In [ ]:
import os
import SimpleITK as sitk
import matplotlib.pyplot as plt
import nibabel as nib
import nibabel.orientations as nio
import numpy as np

# Base dataset directory
# base_dir = "/scratch/awias/data/Totalsegmentator_dataset_v201"
base_dir = "/home/awias/data/Totalsegmentator_dataset_v201"

# Get all subject folders (s0000, s0001, ...)
subjects = sorted([d for d in os.listdir(base_dir) if d.startswith("s")])

idx = 0

subjects = ['s0001']

for subj in subjects:
    ct_path = os.path.join(base_dir, subj, "ct.nii.gz")
    seg_path = os.path.join(base_dir, subj, "segmentations", "vertebrae_L3.nii.gz")

    seg_nib = nib.load(seg_path)
    seg_arr = seg_nib.get_fdata(dtype=np.float32)   # segmentation usually int

    img_nib = nib.load(ct_path)
    arr = img_nib.get_fdata(dtype=np.float32)

    # Get voxel spacing
    zooms = img_nib.header.get_zooms()[:3]

    # Get orientation codes from affine
    ornt = nio.io_orientation(img_nib.affine)
    axcodes = nio.ornt2axcodes(ornt)

    # Build a clean affine: diagonal with voxel spacing
    new_affine = np.diag(list(zooms) + [1])

    # Optionally, you can enforce a pure identity orientation *with voxel spacing*:
    new_affine[:3, :3] = np.eye(3) * np.array(zooms)

    # Reset origin so all images start at the same place
    new_affine[:3, 3] = 0

    # Save new NIfTI with minimal clean header
    new_img = nib.Nifti1Image(arr, affine=new_affine)
    new_seg = nib.Nifti1Image(seg_arr, affine=new_affine)
    # new_img.header.set_xyzt_units('mm')
    # new_seg.header.set_xyzt_units('mm')

    new_orientation = ('L','A','S')  # Desired orientation

    img_nib_reoriented = reorient_to(new_img, axcodes_to=new_orientation)
    seg_nib_reoriented = reorient_to(new_seg, axcodes_to=new_orientation)

    savepath_img = os.path.join(base_dir, subj, "ct_modified.nii.gz")
    savepath_seg = os.path.join(base_dir, subj, "liver_modified.nii.gz")
    nib.save(img_nib_reoriented, savepath_img)
    nib.save(seg_nib_reoriented, savepath_seg)

In [24]:
np.eye(3) * np.array(zooms)

array([[1.49999988, 0.        , 0.        ],
       [0.        , 1.5       , 0.        ],
       [0.        , 0.        , 1.5       ]])

In [23]:
np.diag(list(zooms) + [1])


array([[1.49999988, 0.        , 0.        , 0.        ],
       [0.        , 1.5       , 0.        , 0.        ],
       [0.        , 0.        , 1.5       , 0.        ],
       [0.        , 0.        , 0.        , 1.        ]])

In [12]:
new_affine

array([[1.49999988, 0.        , 0.        , 0.        ],
       [0.        , 1.5       , 0.        , 0.        ],
       [0.        , 0.        , 1.5       , 0.        ],
       [0.        , 0.        , 0.        , 1.        ]])

In [6]:
zooms

(1.5, 1.5000001, 1.5)

In [9]:
zooms = img_nib.header.get_zooms() #Voxel sizes
axs_code = nio.ornt2axcodes(nio.io_orientation(img_nib.affine)) #Image orientation
data_shape_voxels = img_nib.header.get_data_shape() #Shape of data
data_shape_mm = np.array(data_shape_voxels)*np.array(zooms) #Data measures
data_type = img_nib.header.get_data_dtype() #Data type

print("CT Scan Information:")
print(f" - Voxel Sizes (mm): {zooms}")
print(f" - Orientation (RAS): {axs_code}")
print(f" - Data Shape (voxels): {data_shape_voxels}")
print(f" - Data Shape (mm): {data_shape_mm}")
print(f" - Data Type: {data_type}")

CT Scan Information:
 - Voxel Sizes (mm): (np.float32(1.5), np.float32(1.5000001), np.float32(1.5))
 - Orientation (RAS): ('R', 'A', 'S')
 - Data Shape (voxels): (294, 192, 179)
 - Data Shape (mm): [441.         288.00002289 268.5       ]
 - Data Type: int16


In [ ]:
import os
import nibabel as nib
import matplotlib.pyplot as plt

# Base dataset directory
base_dir = "/scratch/awias/data/Totalsegmentator_dataset_v201"

# Get all subject folders (s0000, s0001, ...)
subjects = sorted([d for d in os.listdir(base_dir) if d.startswith("s")])

for subj in subjects:
    ct_path = os.path.join(base_dir, subj, "ct.nii.gz")
    
    if not os.path.exists(ct_path):
        print(f"Skipping {subj}, no CT found.")
        continue

    # Load CT scan with nibabel
    img = nib.load(ct_path)
    arr = img.get_fdata()  # numpy array, shape: (x, y, z)

    # Pick the middle slice in z-direction
    mid_slice = arr.shape[2] // 2
    slice_img = arr[:, :, mid_slice]

    # Plot
    plt.figure(figsize=(5,5))
    plt.imshow(slice_img.T, cmap="gray", origin="lower")
    plt.title(f"{subj} - middle slice")
    plt.axis("off")
    plt.show()
